# THIS IS NOT YET IMPLEMENTED

# Trash bin project

Any city in the world has trash-bins, which fill up over time. Each of the trash bins needs to be emptied from time to time.   
Each workforce in a city work on a certain schedule and therefore on predefined routes to empty trash-bins. The assumption is, that a route always   
follows the same pattern. A person of the workforce will start it's route and approach trash-bins, which have a filling level of 10%, 20% etc.   
Some of the trash-bins are approached and are almost empty. In order to reduce the effort resp. time consumption an optimization should be considered.  

The project idea at hand is to equip each and every trash-bin with an IoT component, which communicates with a central server. The IoT compnent  
should be able to send a timestamp, location- and filling information (e.g. 10, 20, ..., 100%) etc., but also hardware status information about the   
component itself. 

The received information must be processed via big data. A threshold needs to be defined (filling level e.g. 50%), where a trash-bin needs attention.   
The appliaction should monitor the filling levels and propose an optimale route (leeds to an optimization) based on the `travelling salesman problem`.   
A simulated annealing optimization algorithm or an OR - Tools approach should be used to resolve the optimal route of a Garbageman.   






## Context Diagram 

The context diagram shows the main building blocks of the application at hand.

<img src="images/Context_1.jpg" alt="Context Diagram" width="800">

## Scope

- Simulated data (JSON) for Switzerland and Winterthur (any number of trash bin's is ok, but 26 have been used for each)
- Considere filling level of trash-bins and HW maintenance in optimal route
- Optimal Route via Travelling salesman problem 
- Visualization in gmaps with markers and optimal route
- TCP Streaming consumer/producer for Data Analytics
- TCP Client/Server for Route Analytics

Out of scope  
- Historic data 
- Predictive failure detection of HW Module


## Key Performance Indicator (KPI)

- None of the trash bins should be full (consider filling level > 50%)
- hardware failure should also be considered in a route



## Building Blocks of IoT Device Hardware

In many IoT products, the “thing” is fully integrated into the smart device. For example, think of products like an autonomous trash bin. These products control and monitor themselves. In this case, your product includes all four building blocks in a single package as shown below.

<img src="images/iot_building_blocks.jpg" alt="Building Blocks" width="500">


#### Building Block 1: Thing
I define “thing” as the asset that you want to control or monitor.

In many IoT products, the “thing” is fully integrated into the smart device. For example, think of products like an autonomous trash bin. These products control and monitor themselves. In this case, your product includes all four building blocks in a single package as shown below.

#### Building Block 2: Data Acquisition Module
The data acquisition module focuses on acquiring physical signals from the “thing” and converting them into digital signals that can be   
manipulated by a computer.

This is the hardware component that includes all the sensors acquiring real-world signals such as temperature, motion, light, vibration,   
etc. The type and number of sensors you need depend on your application.

#### Building Block 3: Data Processing Module
The third building block of the device is the data processing module. This is the “computer” that processes the data, performs local analytics,   
stores data locally, and performs any other computing operations at the edge.

#### Building Block 4: Communications Module
The last building block of your device’s hardware is the communications module. This is the circuitry that enables communications with your Cloud Platform,   
and with 3rd party systems either locally or in the Cloud.

## Sensor Data / Distance Matrix

### Sensor Data
A Streaming service, as far as I know, does not exist for such kind of data. Therefore, synthetic data needs to be created to simulate the project.   
The data will be structured  in a JSON format as depicted below. A CSV Format could also be considered. An entry contains various fields, but the   
main fields, which will be processed in the appliaction at hand are `ID, Name, Point (lat and lon - coordinates)`, `filling level` and the   
`maintenance flag`.  

<img src="images/image001.jpg" alt="JSON Structure" width="400">

### Sensor Data for Streaming
The data structure sligthly varies in that sense that while streaming the data, each location is applied as a json string in a separate row. 

**Excerpt:**  
{"id": 2,"street": "Street 2","city": "Bern","country": "Switzerland","country_code": "CH","canton": "Bern","canton_code": "BE","lat": 46.948384,"lon": 7.439946,"filling_level": 22,"time": 1610241300,"maintenance": false, "update": false}   
{"id": 3,"street": "Street 3","city": "Luzern","country": "Switzerland","country_code": "CH","canton": "Luzern","canton_code": "LU","lat": 47.055245,"lon": 8.305225,"filling_level": 17,"time": 1610241300,"maintenance": false, "update": false}

### Distance Matrix
`EPSG Geodetic Parameter` Dataset (also EPSG registry) is a public registry of geodetic datums, spatial reference systems, Earth ellipsoids,   
coordinate transformations and related units of measurement. Each entity is assigned an EPSG code between 1024-32767. The EPSG is the basis for   
the distance calculation in the project. Finding the optimal way requires aspects from `garph theaory` as `vertices` and `edges`. A distance Matrix   
is required to define the weights of the edges between vertices. 

Calculating the Distance Matrix requires the coordinates (lat, lon) of each location. Since the project at hand requires the distance between two points   
in respect of the avilable streets rather then the distance by air, I used googles Directions- and Distance Matrix API. Distance Matrix calculation is   
part of the `Location_MAP class`.   


### Package Builder
The databricks environment has the little drawback, that additional source code required in a project needs to be installed as python packages.   
So a trash-bin Package `(trashbin-0.0.5-py3-none-any.whl)`, considering all dependencies has been created for that purpose.  

Python `Package_builder` Project is available here: [Package_builder]()



## TCP Services for Simulation

### TCP Client/Server for Simulation
As mentioned at the beginning there is no streaming service available providing the required data. Therefore, I implemented my own `TCP_Server`   
and a respective `TCP_Client`, to simulate the streaming service. The TCP_Server is not highely sophisticated, it's only `single threaded`.   
We save the `multithreaded server` for latter on. 

The `TCP_Server` contains mainly the following methods: 

- handler - handles all incomming requests from the `TCP_Client` and does a triage
- receive - the TCP_Server `receives` changed trash bin data and `updates` them. This is require, when a garbageman emptied a trash bin
- submit  - the TCP_client `requests` actual data of each trash bin from the TCP_Server 
- random  - `generate random values` on the server to get the feeling the trash bins are filled over time
- acknowledge - acknowledge is only to return the respective state from server to client 

The `TCP_Client` contains mainly the following methods: 

- initalize - `initialize` the `server side data set`
- request_sensor_data  - `requests sensor data` from TCP_Server
- update_sensor_data   - `sends changed sensor data` to be updated on the server 
- assign_random_values - request to `assign random values` to the `filling_level`

### TCP Streaming Consumer/Producer for Simulation

TCP_Server and TCP_Client located in mltools.streaming

The `TCP_Streaming_Server` contains the following method: 
- send() - `send` data via TCP from a file

The `TCP_Streaming_Consumer` contains the following method: 
- receive() - `receive` consumes the sent streaming data

### Travelling Salesman Problem (TSP)
The most famous routing problem is the `Traveling Salesman Problem (TSP)`:   
find the shortest route for a salesman who needs to visit customers at different locations and return to the starting point.   
A TSP can be represented by a graph, in which the nodes correspond to the locations, and the edges (or arcs) denote direct   
travel between locations. The distance between any two locations is given by the weight (distance) next to the edge joining them.

#### Simulated annealing 
Simulated annealing is a probabilistic optimization scheme which guarantees convergence to the global minimum given sufficient run time.  
It’s loosely based on the idea of a metallurgical annealing in which a metal is heated beyond its critical temperature and cooled according   
to a specific schedule until it reaches its minimum energy state. This controlled cooling regiment results in unique material properties useful   
for specific applications.

At it’s core, simulated annealing is based on equation (1) which represents the probability of jumping to the next energy level. Within the context   
of simulated annealing, energy level is simply the current value of whatever function that’s being optimized.

$$P(e_c, e_n,T) = exp(-\Delta{E} / T)$$

$e_c$       - energy at current state  
$e_n$       - energy at proposed neighbor site  
$\Delta{E}$ - change in energy between current state and proposed neighbor state  
$T$         - current system temperature  

The main loop for simulated annealing consists of generating neighbor candidates which are just potential solutions which are then randomly accepted   
based on an ever increasingly more stringent threshold

#### Vehicle routing problems  
`Vehicle routing problems` are inherently intractable: the length of time it takes to solve them grows exponentially with the size of the problem.   
For sufficiently large problems, it could take OR-tools (or any other routing software) years to find the optimal solution. As a result, OR-Tools   
sometimes returns solutions that are good, but not optimal.

`OR-Tools` is open source software for combinatorial optimization, which seeks to find the best solution to a problem out of a    
very large set of possible solutions. 

**Note:**    
We should add that there are other solvers, such as Concorde, dedicated to solving very large TSPs to optimality, which surpass `OR-Tools` or   
`Simulated Annealing` in that area. However, OR-Tools provides a better platform for solving more general routing problems that contain   
constraints beyond those of a pure TSP.  
Both approaches have been implemented and are available in the sources, but considering calculation time the OR-Tool has been applied.    



## Vizualization 

### Googlemaps / gmaps
`Google Maps` is a web mapping service developed by Google. It offers satellite imagery, aerial photography, street maps, 360° interactive   
panoramic views of streets, real-time traffic conditions, and route planning for travelling by foot, car, bicycle, air and public transportation. 

`gmaps` is a plugin for Jupyter for embedding `Google Maps` in notebooks. It is designed as a data visualization tool. In the current project  
`googlemaps` has been used to visualize the `maps`, while `gmaps` has been applied to place `markers` and `directions`.

### Analytics Dashboard

- Filling level per location and respective time window (hour)
- HW - Failure over time
- Maximum filling level and location in a given time window 
- Garbage volume of all trash bins when filling level > 50% -> Assumption of car size required (small / medium / large)
- Average amout of trash per time indow (hour)

## Code Section
### Preliminaries

In [1]:
run_on_databricks = False

# Distinguish between Switzerland or Winterthur
country = True

In [2]:

# Spark libraries 
import pyspark
import socket
import time
from datetime import datetime

from urllib.parse import urlparse
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext

import socket, time

# project libraries
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

import matplotlib.pyplot as plt
import seaborn as sns
import json

# Trash_bin main
from mltools.trash_bin import TrashBin
from mltools.config import Configuration

# Maps
import gmaps
import googlemaps
from mltools.map import Map
from mltools.location_map import Location_Map

# Simulated annealing
from mltools.annealing.nodes_generator import Node_Generator
from mltools.annealing.simulated_annealing import Simulated_Annealing

# Route analytis
from mltools.tsp.route_analytics import Route_Analytics 
from mltools.tsp.travelling_salesman import TSP
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

from plotnine import *

if ((run_local == False) and (run_on_local_spark == False) and (run_on_databricks == True)):
    !pip install googlemaps
    !pip install gmaps
    # !!pip install bunch
    # !pip install GDAL
    # !/databricks/python3/bin/python3.7 -m pip install --upgrade pip
    # !pip install jupyter

%matplotlib inline

### Map, location and sensor data pre - processing

In [ ]:
display(dbutils.fs.ls("/FileStore/tables/trash_bin"))

In [ ]:
if country:
  location_data = spark.read.load("/FileStore/tables/trash_bin/sz_locations.csv", format="csv", sep=",", inferSchema="true", header="true")
  dist_matrix   = spark.read.load("/FileStore/tables/trash_bin/sz_dist_matrix.csv", format="csv", sep=",", inferSchema="true", header="true")
  sensor_data   = spark.read.option("multiline", "true").json("/FileStore/tables/trash_bin/sz_sensor_spark.json")
  json_schema = sensor_data.schema
  sensor_data.printSchema()
else: 
  location_data = spark.read.load("/FileStore/tables/trash_bin/wt_locations.csv", format="csv", sep=",", inferSchema="true", header="true")
  dist_matrix   = spark.read.load("/FileStore/tables/trash_bin/wt_dist_matrix.csv", format="csv", sep=",", inferSchema="true", header="true")
  sensor_data   = spark.read.option("multiline", "true").json("/FileStore/tables/trash_bin/wt_sensor_spark.json")
  json_schema = sensor_data.schema
  sensor_data.printSchema()
    
# convert spark dataframe to pandas dataframe  
location_data = location_data.select("*").toPandas()
dist_matrix   = dist_matrix.select("*").toPandas()
sensor_data   = sensor_data.toPandas()

In [ ]:
location_data.head()

In [ ]:
dist_matrix.head()

In [ ]:
column_names = ['id', 'street', 'city', 'country', 'country_code', 'canton', 'canton_code', 'point', 'lat', 'lon', 'filling_level', 'timestamp', 'maintenance']

sensor_data = sensor_data.reindex(columns = column_names)
sensor_data['level_50'] = np.where(sensor_data['filling_level'] >= 50, True, False)
sensor_data.head()

In [ ]:
display(sensor_data)

In [ ]:
# clean current environment
'''
dbutils.fs.rm("/FileStore/tables/trash_bin/sz_locations.csv")
dbutils.fs.rm("/FileStore/tables/trash_bin/sz_dist_matrix.csv")
dbutils.fs.rm("/FileStore/tables/trash_bin/sz_sensor.json")
dbutils.fs.rm("/FileStore/tables/trash_bin/sz_sensor_spark.json")

dbutils.fs.rm("/FileStore/tables/trash_bin/wt_locations.csv")
dbutils.fs.rm("/FileStore/tables/trash_bin/wt_dist_matrix.csv")
dbutils.fs.rm("/FileStore/tables/trash_bin/wt_sensor.json")
dbutils.fs.rm("/FileStore/tables/trash_bin/wt_sensor_spark.json")
'''

dbutils.fs.rm("/FileStore/tables/trash_bin/sz_sensor_sp.json")

In [ ]:
tb = TrashBin()
map = Location_Map(tb.get_config('trash_bin'), location_data)
config = tb.get_config('tsp')


%md
### Sensor data 



In [ ]:
df_sensor = tb.load_sensor_data(sensor_data)
df_sensor


In [ ]:
# fill level >= 50%
sensors = np.where(df_sensor['filling_level'] >= 50)

# maintenance == True
maintenance = np.where(df_sensor['maintenance'] == True)
nodes = set(np.concatenate([sensors[0], maintenance[0]]))
nodes = list(nodes)  # convert to list again
print(nodes)

%md
### Evaluate shortest distance (Travelling Salesman problem)

In [ ]:
tsp = TSP()
data, ref = tsp.create_data_model(dist_matrix, nodes, prod = True)

route_analytics = Route_Analytics()
manager, routing, solution = route_analytics.estimate_route(data)
route, route_distance = tsp.solution(manager, routing, solution)
route_tsp = route_analytics.ref_map(route, ref)

print('Route Distance: {} [m]'.format(route_distance))
print('Route IDs: {}'.format(route))

## Map
### Prepare markers and directions
Markers are location points, which the garbage man needs to approach. Markers come in two flavours "filling level" and "maintenance".   
The location of a trash bin will indicated with a marker, as soon as he exceeds the 50% filling level. Another marker is applied for   
maintenance reasons. Hardware tends to fail over time. Therefore, we also consider the hw maintenance flag, which needs to be considered   
as alocation point on a route.  

In [ ]:
# TSP
print(route_tsp)
locations_tsp = route_tsp

location_markers_tsp = []

for location in route_tsp: 
    location_markers_tsp.append(map.location_point(location))

print(location_markers_tsp)

### Initialize the Base map

In [ ]:
fig = map.plot_base_map()

In [ ]:
print(route_tsp)
fig = map.add_marker(fig, location_markers_tsp)

length = len(route_tsp)
for i, location in enumerate(location_markers_tsp):
    if i < (length - 1):
        fig = map.add_direction(fig, location, location_markers_tsp[i + 1])

fig

### General purpose Methods

In [ ]:
def getMasterNodeIP():
    '''
    get the IP from the master node
    '''
    try:
        
        url_parsed = urlparse(sc.uiWebUrl)
        ip = url_parsed.netloc.split(':')[0]
        return ip
    
    except Exception as exp:
        print(exp)
        return None

## Data Stream Producer

This notebook provides some basic functionality which one needs to let a stream run so that the Spark Streaming Engine can process it. The stream is produced from a file with trash bin information. In order to simulate streaming data, the producer reads this file continuously and adds a timestamp. 

In [ ]:

def send(inputFile):
  
  ifile  = open(inputFile, 'r') 
  i = 0
  
        
  while True:
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    client_socket.bind((getMasterNodeIP(), portNumber))
    client_socket.listen(10)
    conn, addr = client_socket.accept()

    try:  
      while True:
        start = time.time()
        ifile  = open(inputFile, 'r') 
        for row in ifile.readlines() :

          print("sending: "+ row.rstrip() + "," + str(datetime.now()))
          message = row.rstrip() + ',' + str(datetime.now()) + "\n"
          message = message.encode()
          conn.send(message)

          # send data every 100 ms
          time.sleep(0.1)
                   
    except Exception as e:
        if e.errno == 32:
          # Broken Pipe Exception
          print("Socket was closed. Re-initializing")
        else:
          print(str(e))
        conn.close()
        client_socket.close()
        continue
    finally:
        conn.close()
        client_socket.close()
        

In [ ]:
send("/dbfs/FileStore/tables/sensor_data.csv")

## Consumer

In [ ]:

# Read input stream from socket (by default, sockets contain raw strings, which we must then parse in a structured format)
lines = spark.readStream.format("socket")\
  .option("host", getMasterNodeIP())\
  .option("port", portNumber)\
  .load()

# Split the lines by comma (and assign each resulting column a proper name to reflect its contents, using the alias function)
# Note that wherever needed, we must also cast the resulting column to its proper type (e.g. Float for prices, instead of the default String)

separator = ","

structuredStream = lines.select(\
  split(lines.value, separator)[0].cast('Int').alias("id"),\
  split(lines.value, separator)[1].alias("street"),\
  split(lines.value, separator)[2].alias("street_no"),\
  split(lines.value, separator)[3].cast('Int').alias("x_coord"),\
  split(lines.value, separator)[4].cast('Int').alias("y_coord"),\
  split(lines.value, separator)[5].cast('Float').alias("Ion"),\
  split(lines.value, separator)[6].cast('Float').alias("lat"),\
  split(lines.value, separator)[7].cast('Int').alias("filling_level"),\
  split(lines.value, separator)[8].cast('Timestamp').alias("timestamp"),\
  split(lines.value, separator)[9].cast('Bool').alias("maintenance"))

